In [1]:
import os
import asyncio
import httpx
import pandas as pd

# MADE BY VICTOR DUARTE. VDUART10
base_url = "https://www.gsar.ford.com/onlinereports/be/getVehInfoFull"

In [4]:
URL = "https://corp.sts.ford.com/adfs/oauth2/authorize/wia"
params = {
    "client_id": "urn:gsar:clientid:web:prod",
    "response_type": "token id_token",
    "redirect_uri": "https://www.gsar.ford.com/oauth-callback/",
    "resource": "urn:gsar:resource:web:prod",
    "client-request-id": "c79b8f43-a1cb-472c-1627-0080030000ca",
    "RedirectToIdentityProvider": "AD AUTHORITY",
}
response = httpx.get(URL, params=params)
response

<Response [401 Unauthorized]>

In [ ]:
def create_client() -> httpx.AsyncClient:
    """
    Creates a common client for future http requests

    Returns:
        httpx.Client: client with ford proxies
    """
    limits = httpx.Limits(max_connections=8)
    ford_proxy = str(os.getenv("FORD_PROXY"))
    timeout_config = httpx.Timeout(10.0, connect=5.0, pool=4.0)
    proxy_mounts = {
        "http://": httpx.AsyncHTTPTransport(
            proxy=httpx.Proxy(ford_proxy), limits=limits, retries=3
        ),
        "https://": httpx.AsyncHTTPTransport(
            proxy=httpx.Proxy(ford_proxy), limits=limits, retries=3
        ),
    }
    return httpx.AsyncClient(
        timeout=timeout_config,
        mounts=proxy_mounts,
        verify=False,
    )

In [ ]:
async def get_info_by_vin(
    vin: str, client: httpx.AsyncClient, token: str, semaphore: asyncio.Semaphore
):
    keys = ["prodDate", "plant", "FuelType", "FuelTypeEng"]
    retrived = dict.fromkeys(keys, "")
    async with semaphore:
        response = await client.get(
            base_url,
            params={"vin": vin},
            headers={
                "Authorization": f"Bearer {token}",
            },
        )
    data = dict(response.json())
    for key in keys:
        if key in data:
            retrived[key] = str(data.get(key))
    print(retrived)
    return list(retrived.values())

In [ ]:
vins_df = pd.read_excel("./search.xlsx")  # 29750
semaphore = asyncio.Semaphore(70)  # need to test with 70 instances
token = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6ImFSZ2hZU01kbXI2RFZpMTdWVVJtLUJlUENuayJ9.eyJhdWQiOiJ1cm46Z3NhcjpyZXNvdXJjZTp3ZWI6cHJvZCIsImlzcyI6Imh0dHBzOi8vY29ycC5zdHMuZm9yZC5jb20vYWRmcy9zZXJ2aWNlcy90cnVzdCIsImlhdCI6MTcwODk1NTg5NywiZXhwIjoxNzA4OTg0Njk3LCJDb21tb25OYW1lIjoiVkRVQVJUMTAiLCJzdWIiOiJWRFVBUlQxMCIsInVpZCI6InZkdWFydDEwIiwiZm9yZEJ1c2luZXNzVW5pdENvZGUiOiJGU0FNUiIsImdpdmVuTmFtZSI6IlZpY3RvciIsInNuIjoiRHVhcnRlIiwiaW5pdGlhbHMiOiJWLiIsIm1haWwiOiJ2ZHVhcnQxMEBmb3JkLmNvbSIsImVtcGxveWVlVHlwZSI6Ik0iLCJzdCI6IkJBIiwiYyI6IkJSQSIsImZvcmRDb21wYW55TmFtZSI6IklOU1QgRVVWQUxETyBMT0RJIE4gUkVHSU9OQUwgQkFISUEiLCJmb3JkRGVwdENvZGUiOiIwNjY0Nzg0MDAwIiwiZm9yZERpc3BsYXlOYW1lIjoiRHVhcnRlLCBWaWN0b3IgKFYuKSIsImZvcmREaXZBYmJyIjoiUFJEIiwiZm9yZERpdmlzaW9uIjoiUEQgT3BlcmF0aW9ucyBhbmQgUXVhbGl0eSIsImZvcmRDb21wYW55Q29kZSI6IjAwMDE1ODM4IiwiZm9yZE1hbmFnZXJDZHNpZCI6Im1tYWdyaTEiLCJmb3JkTVJSb2xlIjoiTiIsImZvcmRTaXRlQ29kZSI6IjY1MzYiLCJmb3JkVXNlclR5cGUiOiJFbXBsb3llZSIsImFwcHR5cGUiOiJQdWJsaWMiLCJhcHBpZCI6InVybjpnc2FyOmNsaWVudGlkOndlYjpwcm9kIiwiYXV0aG1ldGhvZCI6Imh0dHA6Ly9zY2hlbWFzLm1pY3Jvc29mdC5jb20vd3MvMjAwOC8wNi9pZGVudGl0eS9hdXRoZW50aWNhdGlvbm1ldGhvZC93aW5kb3dzIiwiYXV0aF90aW1lIjoiMjAyNC0wMi0yNlQxMzowNzozNS4yOTVaIiwidmVyIjoiMS4wIn0.r9Zsg9-TBBhgeiNq6Q083QXTQJyB61jprNN8S8pZ5LUrUxDvzBLlH--s9EFM7rCHT7uFcC2Yokn-F-PakY_YHsrLVXCCPsHQvyCYqaPeGHITJjSAVLLOaRavxNSriGrLqrUBEApqm7PvMzt5z1UP8s1YH6qCi7HThoIvWMc6RDClMc7l9X9i7HFmHQZHigYo-KFg8cf-4kH5_abfPaFBpb74U6NUPsZLnQ6KZW0LPfXRT4d97F6EdyDB8Ay5C3umWMZDyrE7qkeK5IneHGPawwHAP-xJr5_pHkmncWumpIr90-ss3L-c3f83gx3W6dhoPfHza9jPnIajr4jpRxyigQ"

async with create_client() as client:
    vins_df[["prodDate", "plant", "FuelType", "FuelTypeEng"]] = await asyncio.gather(
        *(get_info_by_vin(vin, client, token, semaphore) for vin in vins_df["Full VIN"])
    )

vins_df.head()
vins_df.to_csv("results.csv")

In [ ]:
df = pd.read_json("results.json")
vins_df = pd.read_excel("./search.xlsx")  # 29750

df["VIN"] = vins_df["Full VIN"]
df[["VIN", "prodDate", "plant", "FuelType", "FuelTypeEng"]].to_csv(
    "results.csv", index=False
)

In [ ]:
data = pd.DataFrame(data)

date_obj = pd.to_datetime(data["origWarantDate"], format="%d-%b-%Y")
data["origWarantDate"] = date_obj.dt.strftime("%d/%m/%Y")
date_obj = pd.to_datetime(data["prodDate"], format="%d-%b-%Y")
data["prodDate"] = date_obj.dt.strftime("%d/%m/%Y")